# 2 | Partie Modélisation

### 1. Importation des librairies

In [1]:
import pandas as pd

import missingno as msno

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from scipy.stats import zscore
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline


### 2. Importation des données

In [2]:
df = pd.read_csv("dataset_assurance_cleaned.csv", )
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


In [3]:
### Encodage de nos variables qualitatives pour le schéma de corrélations

df_encoded = df.copy()

# Encodage de la variable 'smoker'
encodeur_smoker = LabelBinarizer()
encodeur_smoker.fit(df_encoded['smoker'])
df_encoded['smoker'] = encodeur_smoker.transform(df_encoded['smoker'])

# Encodage de la variable 'sex'
encodeur_sex = LabelBinarizer()
encodeur_sex.fit(df_encoded['sex'])
df_encoded['sex'] = encodeur_sex.transform(df_encoded['sex'])

# Encodage de la variable 'region'
df_encoded = pd.get_dummies(df_encoded.copy(), columns=['region'], prefix=['region'], dtype=int)

# Renommage des colonnes
df_encoded.rename(columns={'sex': 'sex_male', 'smoker': 'smoker_yes'}, inplace=True)


In [4]:
df_encoded

,age,sex_male,bmi,children,smoker_yes,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1332,50,1,30.970,3,0,10600.54830,0,1,0,0
1333,18,0,31.920,0,0,2205.98080,1,0,0,0
1334,18,0,36.850,0,0,1629.83350,0,0,1,0
1335,21,0,25.800,0,0,2007.94500,0,0,0,1


### 3. Les modèles

#### 3. 1. LinearRegression()

In [5]:
# Séparation des variables explicatives et de la variable cible
X = df_encoded.drop(columns='charges')
y = df_encoded['charges']

In [6]:
# Initialisation du modèle de régression linéaire
modelLR = LinearRegression()

In [7]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X["smoker_yes"])

In [8]:
# Entraînement du modèle de régression linéaire
modelLR.fit(X_train, y_train)

LinearRegression()

In [9]:
# Prédiction sur l'ensemble de test
y_pred = modelLR.predict(X_test)

In [10]:
# Evaluation de la performance du modèle

### MSE  - Erreur quadrique moyenne : mesure l'erruer entre les valeurs réelles et les valeurs prédies
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


### R²   - Coefficient de détermination : mesure la proportion de la variance des données qui est expliquée par le modèle
r2 = modelLR.score(X_test, y_test)
print(f"R²: {r2}")


### MAE  - Erreur Absolue Moyenne : la miyenne des erreurs absolues entre les prédictions et les valeurs réelles
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

Mean Squared Error: 25136492.83553787
R²: 0.8265441393970117
Mean Absolute Error: 3496.304504421919


* R² indique que le modèle est relativement performant, expliquant presque 80 % de la variance des données.
* Le MSE suggère que les erreurs sont relativement grandes en moyenne, mais cela peut être acceptable selon la portée de le problème.
* Le MAE montre que, en moyenne, le modèle fait une erreur d'environ 4,043 unités par prédiction.

In [11]:
coefficients_LR = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': modelLR.coef_
})

coefficients_LR.sort_values('Variable')

,Variable,Coefficient
0,age,260.024973
2,bmi,335.179092
3,children,537.516626
5,region_northeast,692.872647
6,region_northwest,276.157775
7,region_southeast,-505.045851
8,region_southwest,-463.984570
1,sex_male,-195.540957
4,smoker_yes,23823.010094


* Variables influentes : Le fait d'être fumeur (smoker_encoded), l'âge (age), et le BMI (bmi) ont des effets 'positifs' significatifs sur la variable cible.
* Variables modérées : Le nombre d'enfants (children) a un effet 'positif', mais plus faible en comparaison avec d'autres variables comme le BMI ou le statut de fumeur.
* Variables moins influentes : Le sexe (sex_encoded) et la région (region_encoded) ont des effets 'négatifs' sur la variable cible.

### 3. 2. Model Lasso

#### 3.2.1 Méthode Standard

In [12]:
# Initialisation et entrainement du Lasso
lasso_model = Lasso(alpha=1.0)
lasso_model.fit(X_train, y_train)

Lasso()

In [13]:
# Prédiction sur l'ensemble du test
y_pred = lasso_model.predict(X_test)

In [14]:
# Evalusation du modèle
mse = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R²: {r2}")

Mean Squared Error: 3496.14252871994
R²: 0.8265798093933763


In [15]:
# Affichage des coefficients
coefficients_lasso = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': lasso_model.coef_
}).sort_values(by='Variable')

coefficients_lasso

,Variable,Coefficient
0,age,260.030579
2,bmi,334.943427
3,children,536.852657
5,region_northeast,1049.729829
6,region_northwest,632.830398
7,region_southeast,-139.298269
8,region_southwest,-98.614536
1,sex_male,-191.236625
4,smoker_yes,23816.140418


#### 3.2.2 StandardScaler

Standardisation : Centrage et réduction pour avoir une moyenne de 0 et un écart-type de 1.
Pour la régression Lasso, la standardisation est souvent recommandée car elle maintient les distributions tout en ajustant l'échelle.

1. **RobustScaler** :
* Sensibilité aux outliers : Résistant (utilise la médiane et l'IQR, écart interquartile).
* Échelle des données : Basée sur l'IQR (interquartile range).
* Quand l'utiliser ? : Adapté aux données contenant des outliers ou des distributions asymétriques.

2. **StandardScaler** :
* Sensibilité aux outliers : Sensible (utilise la moyenne et l’écart-type).
* Échelle des données : Centre les données à une moyenne de 0 et réduit leur écart-type à 1.
* Quand l'utiliser ? : Adapté aux données sans outliers, ou lorsque les données suivent une distribution normale.

3. **MinMaxScaler** :
* Sensibilité aux outliers : Très sensible (utilise les valeurs min et max).
* Échelle des données : Transforme les données dans une plage définie, généralement [0, 1].
* Quand l'utiliser ? : Utile lorsque les données doivent être dans une plage spécifique (par exemple pour les réseaux neuronaux ou les algorithmes nécessitant des valeurs normalisées).

In [16]:
# Standardisation

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [17]:
lasso_model = Lasso(alpha=1.0)
lasso_model.fit(X_train_scaled, y_train)

Lasso()

In [18]:
y_pred = lasso_model.predict(X_test_scaled)

In [19]:
# Evalusation du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R²: {r2}")

Mean Squared Error: 25133251.63435247
R²: 0.826566505498149


In [20]:
# Affichage des coefficients
coefficients_lasso = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': lasso_model.coef_
}).sort_values(by='Variable')

coefficients_lasso

,Variable,Coefficient
0,age,3651.164029
2,bmi,2037.546767
3,children,637.435988
5,region_northeast,452.483872
6,region_northwest,271.160943
7,region_southeast,-62.636657
8,region_southwest,-42.306976
1,sex_male,-96.663087
4,smoker_yes,9618.025775


#### 3.2.3 Sélection du meilleur alpha

Utilisation de la validation croisée avec une recherche de grille (GridSearchCV) pour tester différentes valeurs d'alpha et choisir celle qui minimise l'erreur de validation.

In [21]:
# Définition de la plage des valeurs d'alpha à tester
param_grid = {
                'alpha' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                'max_iter': [5000, 10000, 50000],
                'tol': [1e-3, 1e-2]
}


In [22]:
# Effectuer une recherche de grille avec validation croisée 

lasso_gs = GridSearchCV(lasso_model, param_grid, cv=5, n_jobs=-1)

In [23]:
lasso_gs.fit(X_train_scaled, y_train)

/home/utilisateur/Documents/Projets/Regression/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.037e+09, tolerance: 1.298e+08
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Documents/Projets/Regression/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.037e+09, tolerance: 1.298e+08
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=5, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'max_iter': [5000, 10000, 50000],
                         'tol': [0.001, 0.01]})

In [24]:
# Afficher le meilleur alpha et ses performances
print(f"Meilleur alpha trouvé : {lasso_gs.best_params_['alpha']}, {lasso_gs.best_estimator_}")

Meilleur alpha trouvé : 10, Lasso(alpha=10, max_iter=5000, tol=0.01)


In [25]:
# Meilleur modèle
best_lasso_model = lasso_gs.best_estimator_
best_lasso_model 

Lasso(alpha=10, max_iter=5000, tol=0.01)

In [26]:
# Test sur le nouveau modèle ajusté
best_lasso_model.fit(X_train_scaled, y_train)
y_pred = best_lasso_model.predict(X_test_scaled)

# Evalusation du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R²: {r2}")

Mean Squared Error: 25106449.87469867
R²: 0.8267514526312612


In [27]:
# Affichage des coefficients
coefficients_lasso = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': lasso_model.coef_
}).sort_values(by='Variable')

coefficients_lasso

,Variable,Coefficient
0,age,3651.164029
2,bmi,2037.546767
3,children,637.435988
5,region_northeast,452.483872
6,region_northwest,271.160943
7,region_southeast,-62.636657
8,region_southwest,-42.306976
1,sex_male,-96.663087
4,smoker_yes,9618.025775


## 2. Transformation polynomiale

Pour améliorer les performances du modèle de régression (Lasso ou autre), nous pouvons appliquer une transformation polynomiale sur vos variables explicatives. Cela permet de capturer des relations non linéaires entre les variables indépendantes et la variable cible.

#### 2.1.1 Façon Standard

In [28]:
# Pipeline avec PolynomialFeatures
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Génération des polynômes
    ('scaler', StandardScaler()),                                # Standardisation
    ('lasso', Lasso(alpha=10, max_iter=5000))                    # Modèle Lasso
])

In [29]:
# Entraînement du modèle
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)

In [30]:
# Évaluation du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R²: {r2}")

Mean Squared Error: 14286683.77830678
R²: 0.901413890706924


#### 2.1.2 Optimisation des hyperparamètres avec GridSearchCV

In [31]:
# Grille des paramètres

param_grid = {
    'poly__degree': [1, 2, 3],                              # Tester différents degrés de polynômes
    'lasso__alpha': range(1, 100),   # Tester différentes valeurs de régularisation
    'lasso__max_iter': [1000, 5000, 10000]                  
}

In [32]:
# GridSearchCV
model_poly = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
model_poly.fit(X_train, y_train)

/home/utilisateur/Documents/Projets/Regression/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.063e+09, tolerance: 1.298e+07
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Documents/Projets/Regression/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.951e+09, tolerance: 1.397e+07
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Documents/Projets/Regression/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number o

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('poly',
                                        PolynomialFeatures(include_bias=False)),
                                       ('scaler', StandardScaler()),
                                       ('lasso',
                                        Lasso(alpha=10, max_iter=5000))]),
             n_jobs=-1,
             param_grid={'lasso__alpha': range(1, 100),
                         'lasso__max_iter': [1000, 5000, 10000],
                         'poly__degree': [1, 2, 3]})

In [33]:
# Meilleur modèle
best_model_poly = model_poly.best_estimator_
best_model_poly

Pipeline(steps=[('poly', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()), ('lasso', Lasso(alpha=21))])

In [34]:
# Prédictions avec le meilleur modèle
y_pred = best_model_poly.predict(X_test)

# Évaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {model_poly.best_params_}")
print(f"Mean Squared Error: {mse}")
print(f"R²: {r2}")



'''Premier essai --> alpha=10'''

Best Parameters: {'lasso__alpha': 21, 'lasso__max_iter': 1000, 'poly__degree': 2}
Mean Squared Error: 14194072.328947023
R²: 0.9020529615095014


'Premier essai --> alpha=10'

In [35]:
# Initialisation
# Pipeline avec PolynomialFeatures
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Génération des polynômes
    ('scaler', StandardScaler()),                                # Standardisation
    ('lasso', Lasso(alpha=21, max_iter=1000))                    # Modèle Lasso
])

# Entraînement du modèle
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)

# Évaluation du modèle
mse_ridge = mean_squared_error(y_test, y_pred)
r2_ridge = r2_score(y_test, y_pred)

print(f"Mean Squared Error (Ridge): {mse_ridge}")
print(f"R² (Ridge): {r2_ridge}")

Mean Squared Error (Ridge): 14194072.328947023
R² (Ridge): 0.9020529615095014


## 3. Modèle Ridge

In [36]:
# Initialisation
ridge_model = Ridge(alpha=1.0, max_iter=5000)

# Entraînement du modèle sur les données standardisées
ridge_model.fit(X_train_scaled, y_train)

# Prédictions sur l'ensemble de test
y_pred_ridge = ridge_model.predict(X_test_scaled)

# Évaluation du modèle
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print(f"Mean Squared Error (Ridge): {mse_ridge}")
print(f"R² (Ridge): {r2_ridge}")

Mean Squared Error (Ridge): 25135354.017568998
R² (Ridge): 0.8265519978779919


In [37]:
coefficients_ridge = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': ridge_model.coef_
}).sort_values(by='Variable')

coefficients_ridge

,Variable,Coefficient
0,age,3648.649692
2,bmi,2037.110664
3,children,637.999269
5,region_northeast,301.830239
6,region_northwest,121.795898
7,region_southeast,-220.539832
8,region_southwest,-193.394756
1,sex_male,-97.194279
4,smoker_yes,9610.473570


In [38]:
# Paramètres
param_grid_ridge = {
    'alpha': range(1, 100),
    'max_iter': [1000, 5000, 10000]
}

# Initialisation de GridSearchCV
ridge_grid_search = GridSearchCV(
    estimator=Ridge(),
    param_grid=param_grid_ridge,
    cv=5,
    n_jobs=-1
)

# Entraînement de la recherche de grille
ridge_grid_search.fit(X_train_scaled, y_train)

# Meilleur modèle Ridge
best_ridge_model = ridge_grid_search.best_estimator_

# Prédictions avec le meilleur modèle
y_pred_best_ridge = best_ridge_model.predict(X_test_scaled)

# Évaluation du meilleur modèle
mse_best_ridge = mean_squared_error(y_test, y_pred_best_ridge)
r2_best_ridge = r2_score(y_test, y_pred_best_ridge)

print(f"Best Alpha (Ridge): {ridge_grid_search.best_params_['alpha']}")
print(f"Mean Squared Error (Best Ridge): {mse_best_ridge}")
print(f"R² (Best Ridge): {r2_best_ridge}")


'''Premier essai --> alpha=10'''

Best Alpha (Ridge): 7
Mean Squared Error (Best Ridge): 25132707.53647755
R² (Best Ridge): 0.8265702600778259


'Premier essai --> alpha=10'

In [39]:
# Initialisation
ridge_model = Ridge(alpha=7, max_iter=5000)

# Entraînement du modèle sur les données standardisées
ridge_model.fit(X_train_scaled, y_train)

# Prédictions sur l'ensemble de test
y_pred_ridge = ridge_model.predict(X_test_scaled)

# Évaluation du modèle
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print(f"Mean Squared Error (Ridge): {mse_ridge}")
print(f"R² (Ridge): {r2_ridge}")

Mean Squared Error (Ridge): 25132707.53647755
R² (Ridge): 0.8265702600778259
